<a href="https://colab.research.google.com/github/jcmachicaocuf/codigos_sencillos_CD/blob/main/U2_Redes_Convolucionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Definir la arquitectura de la CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)  # Capa convolucional 2D con 1 canal de entrada, 32 canales de salida, tamaño del kernel 3x3 y stride 1
        self.conv2 = nn.Conv2d(32, 64, 3, 1)  # Capa convolucional 2D con 32 canales de entrada, 64 canales de salida, tamaño del kernel 3x3 y stride 1
        self.fc1 = nn.Linear(12*12*64, 128)  # Capa lineal con 12*12*64 unidades de entrada y 128 unidades de salida
        self.fc2 = nn.Linear(128, 10)  # Capa lineal con 128 unidades de entrada y 10 unidades de salida (número de clases)

    def forward(self, x):
        x = torch.relu(self.conv1(x))  # Aplicar la función de activación ReLU después de la primera capa convolucional
        x = torch.max_pool2d(x, 2, 2)  # Aplicar max pooling con una ventana de 2x2 y stride 2
        x = torch.relu(self.conv2(x))  # Aplicar la función de activación ReLU después de la segunda capa convolucional
        x = torch.max_pool2d(x, 2, 2)  # Aplicar max pooling con una ventana de 2x2 y stride 2
        x = x.view(-1, 12*12*64)  # Aplanar la salida de las capas convolucionales antes de pasarla a las capas lineales
        x = torch.relu(self.fc1(x))  # Aplicar la función de activación ReLU después de la primera capa lineal
        x = self.fc2(x)  # No aplicamos una función de activación al final para la clasificación
        return x

In [ ]:
# Configuración de los parámetros
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Transformaciones de datos para preprocesamiento
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir las imágenes PIL a tensores
    transforms.Normalize((0.5,), (0.5,))  # Normalizar los valores de píxeles de las imágenes
])

# Descargar y cargar el conjunto de datos MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Inicializar la CNN
model = SimpleCNN()

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Entrenamiento de la CNN
for epoch in range(num_epochs):
    model.train()  # Modo de entrenamiento
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Imprimir estadísticas
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100}')
            running_loss = 0.0

In [ ]:
# Evaluación de la CNN en el conjunto de datos de prueba
model.eval()  # Modo de evaluación
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')
